In [3]:
import polars as pl

In [4]:
df = pl.read_parquet(
    "../data/SMS_Input_-_Motor_Carrier_Census_Information_20250919.parquet"
)
    
df.columns

['DOT_NUMBER',
 'LEGAL_NAME',
 'DBA_NAME',
 'CARRIER_OPERATION',
 'HM_FLAG',
 'PC_FLAG',
 'PHY_STREET',
 'PHY_CITY',
 'PHY_STATE',
 'PHY_ZIP',
 'PHY_COUNTRY',
 'MAILING_STREET',
 'MAILING_CITY',
 'MAILING_STATE',
 'MAILING_ZIP',
 'MAILING_COUNTRY',
 'TELEPHONE',
 'FAX',
 'EMAIL_ADDRESS',
 'MCS150_DATE',
 'MCS150_MILEAGE',
 'MCS150_MILEAGE_YEAR',
 'ADD_DATE',
 'OIC_STATE',
 'NBR_POWER_UNIT',
 'DRIVER_TOTAL',
 'RECENT_MILEAGE',
 'RECENT_MILEAGE_YEAR',
 'VMT_SOURCE_ID',
 'PRIVATE_ONLY',
 'AUTHORIZED_FOR_HIRE',
 'EXEMPT_FOR_HIRE',
 'PRIVATE_PROPERTY',
 'PRIVATE_PASSENGER_BUSINESS',
 'PRIVATE_PASSENGER_NONBUSINESS',
 'MIGRANT',
 'US_MAIL',
 'FEDERAL_GOVERNMENT',
 'STATE_GOVERNMENT',
 'LOCAL_GOVERNMENT',
 'INDIAN_TRIBE',
 'OP_OTHER']

In [5]:
df.shape

(2091643, 42)

In [6]:
addrs_df = pl.read_parquet("../data/geocoded_addresses.parquet")

addrs_df.shape

(2089645, 9)

In [7]:
daxle_matches = []
for i in range(0, 10001, 1000):
    part_df = pl.read_parquet(f"../data/data_axle_matched_addresses/{i}_data.parquet")
    daxle_matches.append(part_df)

daxle_df = pl.concat(daxle_matches).with_columns(
    pl.col("data_axle_row_index")
      .replace(-1, None)
      .alias("data_axle_row_index")
)

In [8]:
daxle_df["confidence"].describe()

statistic,value
str,f64
"""count""",51000.0
"""null_count""",0.0
"""mean""",38.429581
"""std""",46.044908
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",94.871795
"""max""",100.0


In [9]:
daxle = pl.scan_parquet(
    "../data/data-axle.parquet"
).with_row_index(name="data_axle_row_index")


In [ ]:
attempted_matches = daxle_df.filter(pl.col("data_axle_row_index").is_not_null())


indices = attempted_matches.select("data_axle_row_index").to_series()

daxle_subset = (
    daxle
    .collect()
    .head(indices.max() + 1)
)

daxle_subset = daxle_subset.gather(indices)

result = attempted_matches.with_columns(daxle_subset)

In [21]:
joined = daxle_df.lazy().join(daxle, on="data_axle_row_index", how="left")

In [ ]:
daxle_df = joined.collect()

In [ ]:
daxle_df.shape